In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot")
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot")

In [4]:
code = "def hello_world():\n    print('Hello, World!')\n"
prompt = "def hello_world():"
code2 = """\"\"\"Provides utilities to preprocess images for the Inception networks.\"\"\"

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from tensorflow.python.ops import control_flow_ops

def distort_color(image, color_ordering=0, fast_mode=True, scope=None):
  \"\"\"Distort the color of a Tensor image.

Each color distortion is non-commutative and thus ordering of the color ops
matters. Ideally we would randomly permute the ordering of the color ops.
Rather then adding that level of complication, we select a distinct ordering
of color ops for each preprocessing thread.

Args:
image: 3-D Tensor containing single image in [0, 1].
color_ordering: Python int, a type of distortion (valid values: 0-3).
fast_mode: Avoids slower ops (random_hue and random_contrast)
scope: Optional scope for name_scope.
Returns:
3-D Tensor color-distorted image on range [0, 1]
Raises:
ValueError: if color_ordering not in [0, 3]
\"\"\"
with tf.name_scope(scope, 'distort_color', [image]):
  if color_ordering == 0:
    image = tf.image.random_brightness(image, max_delta=32. / 255.)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
  elif color_ordering == 1:
    image = tf.image.random_brightness(image, max_delta=32. / 255.)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=0.2)
  elif color_ordering == 2:
    image = tf.image.random_brightness(image, max_delta=32. / 255.)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=0.2)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
  elif color_ordering == 3:
    image = tf.image.random_brightness(image, max_delta=32. / 255.)
    image = tf.image.random_hue(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
  else:
    raise ValueError('color_ordering must be in [0, 3]')

  if fast_mode:
    image = tf.clip_by_value(image, 0.0, 1.0)
  return image"""

prompt2 = """\"\"\"Provides utilities to preprocess images for the Inception networks.\"\"\"

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from tensorflow.python.ops import control_flow_ops

def distort_color(image, color_ordering=0, fast_mode=True, scope=None):
 \"\"\"Distort the color of a Tensor image.

Each color distortion is non-commutative and thus ordering of the color ops
matters. Ideally we would randomly permute the ordering of the color ops.
Rather then adding that level of complication, we select a distinct ordering
of color ops for each preprocessing thread.

Args:
image: 3-D Tensor containing single image in [0, 1].
color_ordering: Python int, a type of distortion (valid values: 0-3).
fast_mode: Avoids slower ops (random_hue and random_contrast)
scope: Optional scope for name_scope.
Returns:
3-D Tensor color-distorted image on range [0, 1]
Raises:
ValueError: if color_ordering not in [0, 3]
\"\"\""""

In [3]:
def to_tokens_and_probs(model, tokenizer, input_texts, as_log_probs=False):
    input_ids = tokenizer(input_texts, padding=True, return_tensors="pt").input_ids
    outputs = model(input_ids)
    if as_log_probs:
        probs = torch.log_softmax(outputs.logits, dim=-1).detach()
    else:
        probs = torch.softmax(outputs.logits, dim=-1).detach()

    # collect the probability of the generated token -- probability at index 0 corresponds to the token at index 1
    probs = probs[:, :-1, :]
    input_ids = input_ids[:, 1:]
    gen_probs = torch.gather(probs, 2, input_ids[:, :, None]).squeeze(-1)

    batch = []
    for input_sentence, input_probs in zip(input_ids, gen_probs):
        text_sequence = []
        for token, p in zip(input_sentence, input_probs):
            if token not in tokenizer.all_special_ids:
                text_sequence.append((tokenizer.decode(token), p.item()))
        batch.append(text_sequence)
    return batch

In [12]:
import evaluate

perplexity = evaluate.load("perplexity", module_type="metric")

KeyboardInterrupt: 

In [61]:
perplexity.compute(model_id="codeparrot/codeparrot", predictions=[code2, code])

100%|██████████| 1/1 [00:18<00:00, 18.87s/it]


{'perplexities': [1.5722618103027344],
 'mean_perplexity': np.float64(1.5722618103027344)}

In [5]:
from transformers import GenerationConfig
from perplexity import Perplexity

perplexity = Perplexity(model=model, tokenizer=tokenizer, device='cpu', use_chat_template=bool,
                        generation_config=GenerationConfig(
                            # stop_strings],  # Stop generation upon encountering two consecutive newlines
                            max_new_tokens=512,  # Maximum number of tokens to generate
                            pad_token_id=tokenizer.eos_token_id,
                            tokenizer=tokenizer,
                            use_cache=False
                        ), batch_size=2
                        )

In [6]:
test = perplexity._compute([prompt, prompt2], [code, code2], add_start_token=True)

100%|██████████| 1/1 [07:50<00:00, 470.46s/it]
